In [203]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [204]:
tf.__version__

'2.16.1'

In [205]:
dataset = pd.read_csv('Churn_Modelling.csv')
d = dataset.copy()

In [206]:
d

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [207]:
d = d.drop(columns=['RowNumber', 'CustomerId', 'Surname'])
d

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [208]:
d.nunique().sort_values(ascending=False)

EstimatedSalary    9999
Balance            6382
CreditScore         460
Age                  70
Tenure               11
NumOfProducts         4
Geography             3
Gender                2
HasCrCard             2
IsActiveMember        2
Exited                2
dtype: int64

In [209]:
X = d.drop(columns='Exited')
y = d['Exited']

In [210]:
X.nunique().sort_values(ascending=False)

EstimatedSalary    9999
Balance            6382
CreditScore         460
Age                  70
Tenure               11
NumOfProducts         4
Geography             3
Gender                2
HasCrCard             2
IsActiveMember        2
dtype: int64

In [211]:
selected_cols_ = X.columns[X.nunique() < 70].tolist()
print(selected_cols_)

['Geography', 'Gender', 'Tenure', 'NumOfProducts', 'HasCrCard', 'IsActiveMember']


In [212]:
for col in selected_cols_:
    print(d[col].value_counts(ascending=False))
    print()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

Gender
Male      5457
Female    4543
Name: count, dtype: int64

Tenure
2     1048
1     1035
7     1028
8     1025
5     1012
3     1009
4      989
9      984
6      967
10     490
0      413
Name: count, dtype: int64

NumOfProducts
1    5084
2    4590
3     266
4      60
Name: count, dtype: int64

HasCrCard
1    7055
0    2945
Name: count, dtype: int64

IsActiveMember
1    5151
0    4849
Name: count, dtype: int64



In [213]:
X = pd.get_dummies(X, columns=selected_cols_, drop_first=True)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CreditScore        10000 non-null  int64  
 1   Age                10000 non-null  int64  
 2   Balance            10000 non-null  float64
 3   EstimatedSalary    10000 non-null  float64
 4   Geography_Germany  10000 non-null  bool   
 5   Geography_Spain    10000 non-null  bool   
 6   Gender_Male        10000 non-null  bool   
 7   Tenure_1           10000 non-null  bool   
 8   Tenure_2           10000 non-null  bool   
 9   Tenure_3           10000 non-null  bool   
 10  Tenure_4           10000 non-null  bool   
 11  Tenure_5           10000 non-null  bool   
 12  Tenure_6           10000 non-null  bool   
 13  Tenure_7           10000 non-null  bool   
 14  Tenure_8           10000 non-null  bool   
 15  Tenure_9           10000 non-null  bool   
 16  Tenure_10          1000

In [214]:
bool_cols = [col for col in X.columns if X[col].dtype == 'bool']
X[bool_cols] = X[bool_cols].astype(int)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CreditScore        10000 non-null  int64  
 1   Age                10000 non-null  int64  
 2   Balance            10000 non-null  float64
 3   EstimatedSalary    10000 non-null  float64
 4   Geography_Germany  10000 non-null  int32  
 5   Geography_Spain    10000 non-null  int32  
 6   Gender_Male        10000 non-null  int32  
 7   Tenure_1           10000 non-null  int32  
 8   Tenure_2           10000 non-null  int32  
 9   Tenure_3           10000 non-null  int32  
 10  Tenure_4           10000 non-null  int32  
 11  Tenure_5           10000 non-null  int32  
 12  Tenure_6           10000 non-null  int32  
 13  Tenure_7           10000 non-null  int32  
 14  Tenure_8           10000 non-null  int32  
 15  Tenure_9           10000 non-null  int32  
 16  Tenure_10          1000

In [215]:
X.head()

,CreditScore,Age,Balance,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male,Tenure_1,Tenure_2,Tenure_3,...,Tenure_6,Tenure_7,Tenure_8,Tenure_9,Tenure_10,NumOfProducts_2,NumOfProducts_3,NumOfProducts_4,HasCrCard_1,IsActiveMember_1
0,619,42,0.00,101348.88,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1
1,608,41,83807.86,112542.58,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,502,42,159660.80,113931.57,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
3,699,39,0.00,93826.63,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,850,43,125510.82,79084.10,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1


In [216]:
cols2scale = X.iloc[:, :4].columns.tolist()

In [226]:
cols2scale

['CreditScore', 'Age', 'Balance', 'EstimatedSalary']

In [217]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X[cols2scale] = scaler.fit_transform(X[cols2scale])

In [218]:
X

,CreditScore,Age,Balance,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male,Tenure_1,Tenure_2,Tenure_3,...,Tenure_6,Tenure_7,Tenure_8,Tenure_9,Tenure_10,NumOfProducts_2,NumOfProducts_3,NumOfProducts_4,HasCrCard_1,IsActiveMember_1
0,0.538,0.324324,0.000000,0.506735,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1
1,0.516,0.310811,0.334031,0.562709,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0.304,0.324324,0.636357,0.569654,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
3,0.698,0.283784,0.000000,0.469120,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,1.000,0.337838,0.500246,0.395400,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.842,0.283784,0.000000,0.481341,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,1,0
9996,0.332,0.229730,0.228657,0.508490,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,1
9997,0.718,0.243243,0.000000,0.210390,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
9998,0.844,0.324324,0.299226,0.464429,1,0,1,0,0,1,...,0,0,0,0,0,1,0,0,1,0


In [219]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [220]:
# Initializing the ANN
ann = tf.keras.models.Sequential([
    
    # The first hidden layer. ReLU allows only non-negative values to pass through the layer.
    tf.keras.layers.Dense(units=7, activation='relu'),
    
    # The second hidden layer
    tf.keras.layers.Dense(units=7, activation='relu'),
    
    # The third hidden layer
    tf.keras.layers.Dense(units=7, activation='relu'),
    
    # Output layer
    tf.keras.layers.Dense(units=1, activation='sigmoid')

])

In [221]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [222]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)
loss, accuracy = ann.evaluate(X_test, y_test)
print()
print("Test Loss:", round(loss, 2)*100, "%")
print("Test Accuracy:", round(accuracy, 2)*100, "%")

Epoch 1/100


219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 436us/step - accuracy: 0.7717 - loss: 0.5820
Epoch 2/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 408us/step - accuracy: 0.7810 - loss: 0.4787
Epoch 3/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 439us/step - accuracy: 0.7944 - loss: 0.4292
Epoch 4/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 413us/step - accuracy: 0.8216 - loss: 0.4111
Epoch 5/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 408us/step - accuracy: 0.8248 - loss: 0.3946
Epoch 6/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 404us/step - accuracy: 0.8415 - loss: 0.3837
Epoch 7/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 413us/step - accuracy: 0.8401 - loss: 0.3721
Epoch 8/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 413us/step - accuracy: 0.8414 - loss: 0.3759
Epoch 9/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 413us/step - accuracy: 0.8435 - loss: 0.3688
Epoch 10/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 408us/step - accuracy: 0.8390 - loss: 0.3716
Epoch 11/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 410us/step - accuracy: 0.8491 - loss: 0.3609
Epoch 12/100
219/219 ━━━━━━━━━━

In [223]:
history = ann.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

# Access the loss values during training
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Print the last loss value
print("Last Training Loss:", train_loss[-1])
print("Last Validation Loss:", val_loss[-1])

Epoch 1/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 759us/step - accuracy: 0.8705 - loss: 0.3187 - val_accuracy: 0.8590 - val_loss: 0.3545
Epoch 2/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 653us/step - accuracy: 0.8574 - loss: 0.3393 - val_accuracy: 0.8570 - val_loss: 0.3558
Epoch 3/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 665us/step - accuracy: 0.8587 - loss: 0.3311 - val_accuracy: 0.8530 - val_loss: 0.3609
Epoch 4/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 642us/step - accuracy: 0.8718 - loss: 0.3228 - val_accuracy: 0.8537 - val_loss: 0.3562
Epoch 5/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 642us/step - accuracy: 0.8641 - loss: 0.3276 - val_accuracy: 0.8560 - val_loss: 0.3557
Epoch 6/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 667us/step - accuracy: 0.8600 - loss: 0.3393 - val_accuracy: 0.8597 - val_loss: 0.3549
Epoch 7/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 642us/step - accuracy: 0.8584 - loss: 0.3365 - val_accuracy: 0.8570 - val_loss: 0.3557
Epoch 8/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 647us/step - accuracy: 0.8621 - loss: 0.3293 - 

- Both loss values are relatively low, indicating the model might be learning well.
- Validation loss is slightly higher than training loss, suggesting the model is not overfitting severely.

In [225]:
ann.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_18 (Dense)                │ (None, 6)              │           138 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 6)              │            42 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 1)              │             7 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 563 (2.20 KB)

 Trainable params: 187 (748.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 376 (1.47 KB)

#### Prediciton

In [233]:
prediction = ann.predict(X_test)
predicted_churn = prediction > 0.5  
print(predicted_churn)

94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 409us/step
[[ True]
 [False]
 [False]
 ...
 [False]
 [ True]
 [False]]
